In [71]:
# this is my first real python work for my project 'House Prices: Advanced Regression Techniques -  Dubai Edition'
# this scrapper goes through https://www.propertyfinder.ae/ and scraps the property details and save them into CSV

# i've used selenium and beautifulsoup for this purpose. i hope it's not unethical to use both :D

# we scrap data to https://www.propertyfinder.ae/ 
# importing whats needed
from selenium.common.exceptions import NoSuchElementException
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time
import re
import os
from os import path
import urllib3
import random
import requests

from webdriver_manager.firefox import GeckoDriverManager
from random import choice



In [72]:
def proxy_generator():
    response = requests.get("https://sslproxies.org/")
    soup = BeautifulSoup(response.content, 'html5lib')
    proxy = {'https': choice(list(map(lambda x:x[0]+':'+x[1], list(zip(map(lambda x:x.text, soup.findAll('td')[::8]), map(lambda x:x.text, soup.findAll('td')[1::8]))))))}
  
    return proxy
#proxy_generator()

In [73]:

# function to scrap property details page
# parameters: list containing all property details page
# returns: dataframe containing property data
 
def get_property_details(list_property_urls):

    #print('URLs collected:', len(list_property_urls))

    # initializing list for property data
    properties = []
    browser = webdriver.Firefox(executable_path=GeckoDriverManager().install())
    counter = 1
    
    # looping through property urls
    try:    
        for prop_url in list_property_urls:
            print('that is Link {}'.format(prop_url))
            print('\n')
            print('number of element of properties is '+str(len(properties)))
            proxy = proxy_generator()
            #browser = webdriver.Firefox(executable_path=GeckoDriverManager().install(), proxy=proxy)
            #browser = webdriver.Firefox(executable_path = 'C:\\Users\\KDO INFO TECH\\.wdm\\drivers\\geckodriver\\win64\\0.31\\geckodriver.exe', proxy=proxy)
            
            #print("Proxy currently being used: {}".format(proxy))
            
            try:
                print('Getting details of property ', counter, 'of', len(list_property_urls))

                collected_successfully = False

                # setting up variables and objects
                #html = driver.get(prop_url).page_source#urllib3.PoolManager()
                prop_url_parts = prop_url.split('-')
                prop_url_last_part = prop_url_parts[-1]
                property_id = prop_url_last_part.replace('.html', '')
                property_lat = ''
                property_lon = ''
                property_type = ''
                property_size = ''
                property_bedrooms = ''
                property_bathrooms = ''
                property_locality = ''
                property_completion = ''
        
                # checking if the property detail has been captured
                while not collected_successfully:

                    # making http call to get html content
                    #html = http.request('GET', prop_url)
                    
                    # condition to handle a scenario where property is already sold out but is still listed with message
                    # not available
                    # if 'property-gone__area' in html.data.decode('utf-8'):
                    #     collected_successfully = True
                    #     continue

                    # sleeping for 5 second to not attract attention
                    sleep_time = random.randint(5, 8)
                    #time.sleep(sleep_time)

                    # loading property details page html to beautifulsoup
                    
                    browser.get(prop_url)

                    prop_soup = BeautifulSoup(browser.page_source, 'html.parser')
                    #browser.close()
                    #prop_soup = BeautifulSoup(html, 'html.parser')
                    # print('prop_soup')
                    # print(prop_soup)
                    # print('\n')
                    # getting property title
                    property_title = prop_soup.find('h1', {'class': 'property-page__title'}).text
                    # print('property_title')
                    # print(property_title)
                    # print('\n')

                    # getting property price
                    prop_price_div = prop_soup.find('div', {'class': 'property-price__price'})
                    property_price = prop_price_div.text
                    # print('property_price')
                    # print(property_price)
                    # print('\n')
                    property_price = property_price.replace('AED\n', '').strip()
                    property_price = property_price.replace(',', '')

                    # getting locality of the property
                    property_locality_div = prop_soup.find('div', {'class': 'property-location__detail-area'})
                    # print('property_locality_div')
                    # print(property_locality_div)
                    # print('\n')
                    property_locality_div = property_locality_div.find('div', {'class': 'text text--size3'})
                    # print('property_locality_div')
                    # print(property_locality_div)
                    # print('\n')
                    if property_locality_div is not None:
                        property_locality = property_locality_div.text

                    # getting property latitude and longitude details
                    pattern = r'{"lat":[-+]?[0-9]*\.?[0-9]*,"lon":[-+]?[0-9]*\.?[0-9]*}'
                    property_lat_long = re.search(pattern, prop_soup.decode('utf-8'))

                    if property_lat_long is not None:
                        property_location = property_lat_long[0]
                        property_location = property_location.replace('{"lat":', '')
                        property_location = property_location.replace('"lon":', ' ')
                        property_location = property_location.replace('}', '')
                        property_lat = property_location.split(', ')[0]
                        property_lon = property_location.split(', ')[1]

                    # getting list property facts
                    property_facts = prop_soup.find_all('li', {'class': 'property-facts__item'})
                    # print('property_facts')
                    # print(property_facts)
                    # print('\n')
                    if len(property_facts) > 0:
                        # looping list of property facts
                        for property_fact in property_facts:
                            # print(property_fact)
                            # print('\n')
                            
                            # print(property_fact.contents[0].attrs)
                            # print('\n')
                            if len(property_fact.contents[0].attrs) < 2:
                                fact_name = property_fact.contents[0].text.strip()
                                fact_value = property_fact.contents[1].text.strip()

                                if fact_name == 'Completion:':
                                    property_completion = fact_value

                                if fact_name == 'Property type:':
                                    property_type = fact_value

                                if fact_name == 'Property size:':
                                    property_size = re.sub(r'[^\w\s\t\xa0]','',fact_value)
                                    property_size = property_size.replace('\t','')
                                    property_size = property_size.replace('\n','')
                                    property_size = property_size.replace('\xa0',' ')

                                if fact_name == 'Bedrooms:':
                                    property_bedrooms = fact_value.replace('\n','')
                                    property_bedrooms = property_bedrooms.replace('\t','')
                                    property_bedrooms = property_bedrooms.replace('Maid','')
                                    property_bedrooms = property_bedrooms.replace('+','')

                                if fact_name == 'Bathrooms:':
                                    property_bathrooms = fact_value
                    # else:
                    #     property_facts = prop_soup.find_all('p', {'class': 'listing-info__item-title'})
                    #     for obj in property_facts :   
                    #         if 'bedrooms' in obj.text: 
                    #             property_bedrooms = obj.text
                    #         if 'bathrooms' in obj.text:
                    #             property_bathrooms = obj.text
                    #         if 'sqft' in obj.text:
                    #             property_size = obj.text
                    # getting property amenities and converting them to comma separated string
                    prop_amen = ''
                    property_amenities = prop_soup.find_all('div', {'class': 'property-amenities__list'})
                    if len(property_amenities) > 0:
                        for property_amenity in property_amenities:
                            prop_amen += property_amenity.text.strip() + ', '

                    # storing property detail into multidimensional array
                    properties.append({'id': property_id,
                                    'title': property_title,
                                    'price': property_price,
                                    'location': property_locality,
                                    'latitude': property_lat,
                                    'longitude': property_lon,
                                    'type': property_type,
                                    'size': property_size,
                                    'no_of_bedrooms': property_bedrooms,
                                    'no_of_bathrooms': property_bathrooms,
                                    'completion_status': property_completion,
                                    'amenities': prop_amen})

                    # marking property details collected successfully
                    collected_successfully = True
                    
                    print('properties')
                    print(properties)
                    #browser.close()
                    counter += 1
            except:
                continue
    except:
        #browser.close()
        return pd.DataFrame(properties)        
    # returning dataframe
    #browser.close()
    return pd.DataFrame(properties)


In [74]:

# function to get property urls
# parameters: number of properties
# returns: list containing urls
def get_properties_url(num_of_properties):

    # initializing the web driver
    options = webdriver.ChromeOptions()

    # change the path to where chromedriver is in your home folder.
    driver = webdriver.Chrome(
        executable_path="D:\\AshDownX\\chromedriver.exe",
        options=options)
    driver.set_window_size(1120, 1000)

    # base url to start from. you can change parameters by opening the below url in browser
    # and then modify the onscreen filters, and then copy-paste the updated url below
   # url = 'https://famproperties.com/palm-jumeirah-dubai/emerald-apartment/1-bedroom-Apartment-for-Sale-72920'#'https://www.xxxxxxxx.xx/en/search?c=1&l=1&ob=mr&page=1&t=1'
    url = 'https://www.propertyfinder.ae/en/search?c=1&ob=nd&page=#&t=1'

    property_urls = []
    
    options = webdriver.ChromeOptions()

    # change the path to where chromedriver is in your home folder.
    driver = webdriver.Chrome(  executable_path="D:\\AshDownX\\chromedriver.exe", options=options)    
    #driver.set_window_size(1120, 1000)

   
    #print(soup)
    #elt = soup.find('div',class_='card-list__item')
    #print(driver)
    # looping until the number of property
    n = 60
    try:    
        while n <= num_of_properties: #num_of_properties is number of page 
            driver.get(url.replace('#',''+str(n)))
            soup=BeautifulSoup(driver.page_source, 'html.parser')
        
            time.sleep(5)
            property_cards = soup.find_all('div',class_="card-list__item")
            # print('property_cards')
            # print(property_cards)
            for property_card in property_cards:
                time.sleep(1)

                # checking if there is an ad appearing in the middle of the property listing
                # the first element of a valid property is always an A tag. If not, it's an ad
                link = property_card.find('a')
                if link is not None:
                    link = link.get('href')
                    if link is not None:
                        if not 'https://www.propertyfinder.ae' in link:
                            property_urls.append('https://www.propertyfinder.ae'+str(link))
                            print('https://www.propertyfinder.ae'+str(link))
                        else:
                            property_urls.append(str(link))
                            print(link)
            n = n+1
            print('page = '+str(n))
    except :
        return property_urls
    return property_urls


In [75]:
# function to start the scrapping process and save the csv file
# parameters: number of properties, (filename including .csv extension)
def scrap_properties(num_of_properties, filename):

    # urls = get_properties_url(num_of_properties)

    # # saving list of urls to csv. so we don't loose progress after collecting all the urls
    # url_df = pd.DataFrame(urls)
    # url_df.to_csv('urls.csv')

    # calling function to get property details for all the urls. passing list of urls and object selenium driver
    if path.exists('urls.csv'):
        url_df = pd.read_csv('urls.csv')

    df = get_property_details(url_df['0'])

    # calling function to save the extracted dataframe into csv file
    save_data_csv(df, filename)


# function to save dataframe to csv file
# parameters: dataframe, filename (including .csv extension)
def save_data_csv(data_frame, filename):

    # checking if the file exist. delete if it is already there
    if path.exists(filename):
        os.remove(filename)

    # saving to csv. please include .csv extension in filename
    data_frame.to_csv(filename, index=False)


In [76]:

# it begins here. call this function kick start the whole chain
# parameters: number of properties, (filename including .csv extension)
def start_scrapping_properties(num_of_properties, filename):
    # calling function to scrap properties. passing number of properties to be extracted and filename for csv.
    scrap_properties(num_of_properties, filename)



In [77]:
# importing libraries
from datetime import datetime

# printing start time of the operation
print('started at:', datetime.now().strftime("%d/%m/%Y %H:%M:%S"))

# calling function to start the scrapping process and save it in a csv format
start_scrapping_properties(100, 'properties.csv')

# printing end time of the operation
print('completed at:', datetime.now().strftime("%d/%m/%Y %H:%M:%S"))


started at: 07/09/2022 16:48:29


[WDM] - Downloading: 16.2kB [00:00, 4.15MB/s]                   


that is Link https://www.propertyfinder.ae/en/plp/rent/apartment-for-rent-dubai-international-city-emirates-cluster-8885534.html


number of element of properties is 0
Getting details of property  1 of 91
properties
[{'id': '8885534', 'title': 'Iconic Apt w/ Balcony and Community View', 'price': '30000 AED  / yearly', 'location': 'Dubai, International City', 'latitude': '25.161765', 'longitude': '55.422453', 'type': 'Apartment', 'size': '834sqft  77sqm', 'no_of_bedrooms': '1', 'no_of_bathrooms': '2', 'completion_status': '', 'amenities': 'Unfurnished, Balcony, Built in Wardrobes, Central A/C, '}]
that is Link https://www.propertyfinder.ae/en/plp/rent/apartment-for-rent-dubai-business-bay-waves-tower-8898180.html


number of element of properties is 1
Getting details of property  2 of 91
that is Link https://www.propertyfinder.ae/en/plp/rent/apartment-for-rent-dubai-jumeirah-village-circle-five-at-jumeirah-village-circle-8895972.html


number of element of properties is 1
Getting detail